In [1]:
import numpy as np
import os
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt


A module that was compiled using NumPy 1.x cannot be run in
NumPy 2.1.3 as it may crash. To support both 1.x and 2.x
versions of NumPy, modules must be compiled with NumPy 2.0.
Some module may need to rebuild instead e.g. with 'pybind11>=2.12'.

If you are a user of the module, the easiest solution will be to
downgrade to 'numpy<2' or try to upgrade the affected module.
We expect that some modules will need time to support NumPy 2.

Traceback (most recent call last):  File "<frozen runpy>", line 198, in _run_module_as_main
  File "<frozen runpy>", line 88, in _run_code
  File "C:\Users\varsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel_launcher.py", line 18, in <module>
    app.launch_new_instance()
  File "C:\Users\varsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
    app.start()
  File "C:\Users\varsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\ipykernel\kernelapp.py", 

AttributeError: _ARRAY_API not found

In [2]:
# Set paths
train_dir = 'C:\\Users\\varsh\\Downloads\\archive (3)\\data\\train'
val_dir = 'C:\\Users\\varsh\\Downloads\\archive (3)\\data\\validation'

In [3]:
# Get class labels from folder names
class_labels = sorted(os.listdir(train_dir))

In [4]:
# Check class distribution in your training dataset
class_counts = {class_label: 0 for class_label in class_labels}

for class_label in class_labels:
    class_counts[class_label] = len(os.listdir(os.path.join(train_dir, class_label)))

print(f"Class distribution in training data: {class_counts}")

Class distribution in training data: {'Acral_Lentiginous_Melanoma': 735, 'Healthy_Nail': 323, 'Onychogryphosis': 677, 'blue_finger': 603, 'clubbing': 767, 'pitting': 639}


In [5]:
# Image size & batch
IMG_SIZE = (224, 224)
BATCH_SIZE = 32

# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

In [6]:
# Only rescaling for validation
val_datagen = ImageDataGenerator(rescale=1./255)

# Load data
train_data = train_datagen.flow_from_directory(
    train_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical'
)

val_data = val_datagen.flow_from_directory(
    val_dir,
    target_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

Found 3744 images belonging to 6 classes.
Found 91 images belonging to 6 classes.


In [7]:
# Load MobileNetV2 without top layers
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze it initially

# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dense(len(class_labels), activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

In [8]:
# Unfreeze the last 10 layers of MobileNetV2
for layer in base_model.layers[-10:]:  # You can adjust the number of layers you want to unfreeze
    layer.trainable = True

# Compile the model again after unfreezing layers
model.compile(optimizer=Adam(learning_rate=0.00001), loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)


In [10]:
# Define class weights
class_weights = {
    0: 1.0,  # Acral_Lentiginous_Melanoma
    1: 1.5,  # Healthy_Nail (lowest number of samples, give it a higher weight)
    2: 1.0,  # Onychogryphosis
    3: 1.0,  # blue_finger
    4: 1.0,  # clubbing
    5: 1.2   # pitting
}


In [11]:
model.fit(
    train_data,
    validation_data=val_data,
    epochs=50,
    callbacks=[early_stop],
    verbose=1,
    class_weight=class_weights
)


C:\Users\varsh\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 156s 1s/step - accuracy: 0.2898 - loss: 1.7907 - val_accuracy: 0.2967 - val_loss: 1.5904
Epoch 2/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 172s 1s/step - accuracy: 0.5715 - loss: 1.2728 - val_accuracy: 0.3626 - val_loss: 1.4284
Epoch 3/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 149s 1s/step - accuracy: 0.6588 - loss: 1.0425 - val_accuracy: 0.4176 - val_loss: 1.2769
Epoch 4/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 147s 1s/step - accuracy: 0.7066 - loss: 0.9132 - val_accuracy: 0.5055 - val_loss: 1.1146
Epoch 5/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 148s 1s/step - accuracy: 0.7007 - loss: 0.8502 - val_accuracy: 0.6154 - val_loss: 0.9668
Epoch 6/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 153s 1s/step - accuracy: 0.7394 - loss: 0.7554 - val_accuracy: 0.6923 - val_loss: 0.8527
Epoch 7/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 150s 1s/step - accuracy: 0.7380 - loss: 0.7596 - val_accuracy: 0.7363 - val_loss: 0.7648
Epoch 8/50
117/117 ━━━━━━━━━━━━━━━━━━━━ 157s 1s/step - accuracy: 0.7655 - loss: 0.6886 - val_accu

In [12]:
# Evaluate on validation set
val_data.reset()
y_pred = np.argmax(model.predict(val_data), axis=1)
y_true = val_data.classes[:len(y_pred)]

accuracy = accuracy_score(y_true, y_pred)
print(f"\n✅ Validation Accuracy: {accuracy * 100:.2f}%")

3/3 ━━━━━━━━━━━━━━━━━━━━ 10s 3s/step

✅ Validation Accuracy: 90.11%


In [27]:
model.save("naildisease_model.h5")

In [28]:
image_size = (224, 224)
# Step 7: Related systemic conditions mapping
nail_to_systemic_map = {
    'onycholysis': ['Thyroid disorders', 'Iron-deficiency anemia', 'Psoriasis'],
    'melanonychia': ['Melanoma', 'HIV', 'Laugier–Hunziker syndrome'],
    'pitting': ['Heart disease', 'Lung cancer', 'Inflammatory bowel disease'],
    'koilonychia': ['Iron-deficiency anemia', 'Celiac disease', 'Heart disease'],
    'leukonychia': ['Liver cirrhosis', 'Kidney failure', 'Zinc deficiency'],
    'beau_lines': ['Diabetes', 'High fever', 'Chemotherapy side effects'],
    'yellow_nail_syndrome': ['Respiratory diseases', 'Lymphedema'],
    'muehrcke_lines': ['Hypoalbuminemia', 'Kidney disease'],
    'clubbing': ['Psoriasis', 'Alopecia areata'],
    'healthy': ['No associated systemic disease']
}

# Step 8: Predictor Function
def predict_nail_disease(img_path):
    model = load_model("naildisease_model.h5")

    img = load_img(img_path, target_size=image_size)
    img_array = img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    predicted_index = np.argmax(prediction)
    predicted_class = class_labels[predicted_index]
   
    # Show the image
    plt.imshow(img)
    plt.title(f"Prediction: {predicted_class}")
    plt.axis('off')
    plt.show()
    
    print(f"\n🔍 Predicted Nail Condition: {predicted_class}")
    print("🩺 Possible Related Health Conditions:")
    for disease in nail_to_systemic_map.get(predicted_class, ["No associated condition found."]):
        print(f" - {disease}")

In [ ]:
# Step 9: Example usage
# Replace with your own image path
predict_nail_disease("025.jpg")